<a href="https://colab.research.google.com/github/Hamoda-dabbit/Mining---classification-in-Arabic-Article/blob/main/CNN_%3CUNK%3E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 1- إستيراد المكتبات

In [ ]:
import pandas as pd
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
import nltk
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

--------

2- استيراد ملفات البيانات

In [ ]:
dataset = pd.read_excel("drive/MyDrive/mining/1.xlsx")
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17500 entries, 0 to 17499
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    17361 non-null  object
 1   targe   17500 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 273.6+ KB


------
3- shuffle  
بعثرة البيانات

In [ ]:
from sklearn.utils import shuffle
dataset=shuffle(dataset)
dataset.head()

,text,targe
1411,علمت الصباح أن الفيلم الجديد للمخرج نبيل عيوش ...,0
7555,يستعد المغرب لإطلاق أول دفعة من السندات الإسلا...,2
4930,استقلال القاضي هدف ثابت وحتمي لتحقيق العدالة ف...,1
10054,بلغ حجم الإعفاءات الضريبية خلال العام الحالي م...,2
2714,أخبارنا المغربية سناء الوردي استغلت الفنانة ال...,0


------
4- clean
تنظيف البيانات

In [ ]:
import string
import re
from nltk.stem.isri import ISRIStemmer

print(np.sum(dataset.isnull().any(axis=1)))
dataset= dataset.dropna()
print(np.sum(dataset.isnull().any(axis=1)))

#-------------
def remove_hashtag(df, col = 'text'):
    for letter in r'#.][!XR':
      df[col] = df[col].astype(str).str.replace(letter,'', regex=True)
           
remove_hashtag(dataset)
#-------------
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
#-------------
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text
#-------------    
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)
#-------------
def processDocument(doc, stemmer): 

    #Replace @username with empty string
    doc = re.sub(r'@[^\s]+', ' ', doc)
    doc = re.sub(r'_', ' ', doc)
    doc = re.sub(r'\n', ' ', doc)
    doc = re.sub(r'[a-z,A-Z]', '', doc)
    doc = re.sub(r'\d', '', doc)
    #Convert www.* or https?://* to " "
    doc = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',doc)
    #Replace #word with word
    doc = re.sub(r'#([^\s]+)', r'\1', doc)
    # remove punctuations
    doc= remove_punctuations(doc)
    # normalize the tweet
    doc= normalize_arabic(doc)
    # remove repeated letters
    doc=remove_repeating_char(doc)
    #stemming
    doc = stemmer.stem(doc)
    
    return doc
    
stemmer = ISRIStemmer()
dataset["text"] = dataset['text'].apply(lambda x: processDocument(x, stemmer))


139
0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [ ]:
# build train and test datasets
reviews = dataset['text'].values
sentiments = dataset['targe'].values
# print(y[0])



# from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.2)
# train_reviews = reviews[:35000]
train_reviews = reviews[:14000]
train_sentiments = sentiments[:14000]
# y_cat = y[:14000]

# test_reviews = reviews[35000:]
test_reviews = reviews[14000:]
test_sentiments = sentiments[14000:]
# y_test = y[14000:]

norm_train_reviews = (train_reviews)
norm_test_reviews = (test_reviews)


le = LabelEncoder()
y_train = le.fit_transform(train_sentiments)
y_test = le.transform(test_sentiments)



# print(y_train)
# y_test = to_categorical(y_test)
# y_cat


#convert text to numper 

num_classes=5 #  -> 0,  -> 1 , 2, 3, ,4

# print(y[0])
#convert numper to array  1=>[0,1,0,0,0]
#Converting prediction to categorical
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
# print(y[0])

## Preprocessing

To prepare text data for our deep learning model, we transform each review into a sequence.
Every word in the review is mapped to an integer index and thus the sentence turns into a sequence of numbers.

To perform this transformation, keras provides the ```Tokenizer```

In [ ]:
t = Tokenizer(oov_token='<UNK>')
# fit the tokenizer on the documents
t.fit_on_texts(norm_train_reviews)
t.word_index['<PAD>'] = 0
# max([(k, v) for k, v in t.word_index.items()], key = lambda x:x[1]), min([(k, v) for k, v in t.word_index.items()], key = lambda x:x[1]), t.word_index['<UNK>']
train_sequences = t.texts_to_sequences(norm_train_reviews)
test_sequences = t.texts_to_sequences(norm_test_reviews)
print("Vocabulary size={}".format(len(t.word_index)))
print("Number of Documents={}".format(t.document_count))


Vocabulary size=160097
Number of Documents=14000


### Sequence Normalization

Not all reviews are of same length. To handle this difference in length of reviews, we define a maximum length.
For reviews which are smaller than this length, we pad them with zeros which longer ones are truncated

In [ ]:
MAX_SEQUENCE_LENGTH = 1000
# pad dataset to a maximum review length in words
X_train = sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
X_test = sequence.pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
X_train.shape, X_test.shape

((14000, 1000), (3361, 1000))

### Encoding Labels

The dataset contains labels of the form positive/negative. The following step encodes the labels using ```sklearn's``` ```LabelEncoder```

## Prepare the Model

Since textual data is a sequence of words, we utilize ```1D``` convolutions to scan through the sentences.
The model first transforms each word into lower dimensional embedding/vector space followed by 1d convolutions and then passing the data through dense layers before the final layer for classification

In [ ]:
VOCAB_SIZE = len(t.word_index)
EMBED_SIZE = 300
EPOCHS=2
BATCH_SIZE=128
# create the model
model = Sequential()
model.add(Embedding(VOCAB_SIZE, EMBED_SIZE, input_length=MAX_SEQUENCE_LENGTH))
model.add(Conv1D(filters=128, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))

# model.add(Dense(1, activation='sigmoid'))
model.add(Dense(5, activation='sigmoid'))
# model.add(Flatten())
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1000, 300)         48029100  
                                                                 
 conv1d_3 (Conv1D)           (None, 1000, 128)         153728    
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 500, 128)         0         
 1D)                                                             
                                                                 
 conv1d_4 (Conv1D)           (None, 500, 64)           32832     
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 250, 64)          0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 250, 32)          

## Model Training

In [ ]:
# Fit the model
model.fit(X_train, y_train, 
          validation_split=0.1,
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          verbose=1)

Epoch 1/2
99/99 [==============================] - 367s 4s/step - loss: 0.3095 - accuracy: 0.5965 - val_loss: 0.0755 - val_accuracy: 0.9357
Epoch 2/2
99/99 [==============================] - 360s 4s/step - loss: 0.0383 - accuracy: 0.9717 - val_loss: 0.0585 - val_accuracy: 0.9571


## Model Evaluation

In [ ]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

106/106 [==============================] - 22s 210ms/step - loss: 0.0769 - accuracy: 0.9458
Accuracy: 94.58%


In [35]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def calculate_results(y_true, y_pred):
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [36]:
y_pred=(model.predict(X_test))
y_classes_pre = [np.argmax(y, axis=None, out=None) for y in y_pred]
y_classes_test = [np.argmax(y, axis=None, out=None) for y in y_test]

result_CNN=(calculate_results(y_classes_test, y_classes_pre))
result_CNN

{'accuracy': 94.58494495685808,
 'f1': 0.9460435614590436,
 'precision': 0.9479702608343531,
 'recall': 0.9458494495685807}

---------------


In [ ]:
# predictions = model.predict_classes(X_test).ravel()
# predictions[:10]

In [ ]:
# predictions = ['positive' if item == 1 else 'negative' for item in predictions]
# predictions[:10]

In [ ]:
# from sklearn.metrics import confusion_matrix, classification_report

# labels = ['negative', 'positive']
# print(classification_report(test_sentiments, predictions))
# pd.DataFrame(confusion_matrix(test_sentiments, predictions), index=labels, columns=labels)